### Imports

In [64]:
import pandas as pd
import numpy as np
import openpyxl

### Load Data

In [65]:
df = pd.read_excel(r"data_masterstudie_2022-07-14_13-39.xlsx")

In [66]:
df

,CASE,SERIAL,REF,QUESTNNR,MODE,STARTED,AE01_01,AE01_02,AE02_01,AE03_01,...,MAILSENT,LASTDATA,FINISHED,Q_VIEWER,LASTPAGE,MAXPAGE,MISSING,MISSREL,TIME_RSI,DEG_TIME
0,Interview-Nummer (fortlaufend),Seriennummer (sofern verwendet),Referenz (sofern im Link angegeben),"Fragebogen, der im Interview verwendet wurde",Interview-Modus,Zeitpunkt zu dem das Interview begonnen hat (E...,Rsikoverhalten: Investition in Alternative A,Rsikoverhalten: Investition in Alternative B,Quelle: Erbe,ErbeRiskant: Alternative 2,...,Versandzeitpunkt der Einladungsmail (nur für n...,Zeitpunkt als der Datensatz das letzte mal geä...,Wurde die Befragung abgeschlossen (letzte Seit...,Hat der Teilnehmer den Fragebogen nur angesehe...,"Seite, die der Teilnehmer zuletzt bearbeitet hat","Letzte Seite, die im Fragebogen bearbeitet wurde",Anteil fehlender Antworten in Prozent,Anteil fehlender Antworten (gewichtet nach Rel...,Maluspunkte für schnelles Ausfüllen,Maluspunkte für schnelles Ausfüllen
1,250,NaN,NaN,base,interview,2022-07-06 09:52:30,NaN,NaN,NaN,NaN,...,NaN,2022-07-06 09:56:06,1,0,7,7,3,0,1.8,41
2,253,NaN,NaN,base,interview,2022-07-06 13:27:25,NaN,NaN,NaN,NaN,...,NaN,2022-07-06 13:35:37,1,0,7,7,6,3,0.81,1
3,254,NaN,NaN,base,interview,2022-07-06 13:30:33,NaN,NaN,NaN,NaN,...,NaN,2022-07-06 13:35:32,1,0,7,7,3,0,1.42,21
4,255,NaN,NaN,base,interview,2022-07-06 14:19:42,NaN,NaN,NaN,NaN,...,NaN,2022-07-06 14:33:27,1,0,7,7,71,73,1.22,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185,535,NaN,NaN,base,interview,2022-07-14 11:53:48,NaN,NaN,NaN,NaN,...,NaN,2022-07-14 11:57:50,1,0,7,7,3,0,1.67,33
186,536,NaN,NaN,base,interview,2022-07-14 12:45:21,NaN,NaN,NaN,NaN,...,NaN,2022-07-14 12:51:22,1,0,7,7,3,0,1.15,9
187,537,NaN,NaN,base,interview,2022-07-14 13:11:05,NaN,NaN,NaN,NaN,...,NaN,2022-07-14 13:15:24,1,0,7,7,3,0,1.77,53
188,538,NaN,NaN,base,interview,2022-07-14 13:16:04,NaN,NaN,NaN,NaN,...,NaN,2022-07-14 13:27:32,1,0,7,7,3,0,1.64,69


### Filter for relevant columns

In [67]:
df = df[["FS01_CP","FS01","SD03","SD05","SD11","SD11_10","SD14","SD14_08","SD16","MZ03_01","MZ03_02","MZ03_03","MZ03_04","MZ03_05","MZ03_06","SC01","SC02","VR01_01",
"AN01_01","AN01_02","AN02_01","AN02_02","AN03_01","AN03_02","AN04_01","AN04_02","RI01","RI02","RI03","RI04","RI05","RI06","RI07","RI08","RI09","RI10","RI11","RI12","RI13","TIME_SUM","FINISHED"
]]

In [68]:
df.columns

Index(['FS01_CP', 'FS01', 'SD03', 'SD05', 'SD11', 'SD11_10', 'SD14', 'SD14_08',
       'SD16', 'MZ03_01', 'MZ03_02', 'MZ03_03', 'MZ03_04', 'MZ03_05',
       'MZ03_06', 'SC01', 'SC02', 'VR01_01', 'AN01_01', 'AN01_02', 'AN02_01',
       'AN02_02', 'AN03_01', 'AN03_02', 'AN04_01', 'AN04_02', 'RI01', 'RI02',
       'RI03', 'RI04', 'RI05', 'RI06', 'RI07', 'RI08', 'RI09', 'RI10', 'RI11',
       'RI12', 'RI13', 'TIME_SUM', 'FINISHED'],
      dtype='object')

### Drop first row 

In [69]:
df = df.iloc[1: , :]

### Subtract 1 from some columns because they start at 1 not at 0

In [70]:
df[['AN01_01','AN01_02','AN02_01','AN02_02', 'AN03_01', 'AN03_02', 'AN04_01', 'AN04_02']] = df[['AN01_01','AN01_02','AN02_01','AN02_02', 'AN03_01', 'AN03_02', 'AN04_01', 'AN04_02']] -1

In [71]:
df[['AN01_01','AN01_02','AN02_01','AN02_02', 'AN03_01', 'AN03_02', 'AN04_01', 'AN04_02']]

,AN01_01,AN01_02,AN02_01,AN02_02,AN03_01,AN03_02,AN04_01,AN04_02
1,NaN,NaN,8,92,52,48,82,18
2,NaN,NaN,20,80,20,80,20,80
3,28,72,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,-10,-10,-10,-10,-10,-10
5,50,50,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
185,NaN,NaN,0,100,0,100,50,50
186,30,70,NaN,NaN,NaN,NaN,NaN,NaN
187,NaN,NaN,25,75,18,82,70,30
188,20,80,NaN,NaN,NaN,NaN,NaN,NaN


### Exclude data with short answer time (<3 minutes) and data where no consent/the last page was not reached or a lot of questions were not answered.

In [72]:
df_short_answer = df[df["TIME_SUM"] >= 180 ]

In [73]:
df_finished = df_short_answer[df_short_answer["FINISHED"] == 1] 

In [74]:
df_consent = df_finished[df_finished["SC01"] == 2 ]

In [75]:
df_overall = df_consent[df_consent["SC02"] == 1]

In [76]:
df_final = df_overall.loc[(df_overall["RI01"] != -9)  & (df_overall["RI02"] != -9)]

In [77]:
df_final = df_final.loc[(df_final["AN01_01"] != -10)  & (df_final["AN01_02"] != -10) &(df_final["AN02_01"] != -10) &(df_final["AN02_02"] != -10) &(df_final["AN03_01"] != -10) &(df_final["AN03_02"] != -10) &(df_final["AN04_01"] != -10) &(df_final["AN04_02"] != -10)]

In [78]:
df_final = df_final.reset_index(drop=True)

### Create a Column with the total amount invested in the risky/save alternative

In [79]:

df_final['SAVE_TOTAL']= df_final['AN02_02'].where(df_final['FS01'] ==1, 0) * 0.2 + df_final['AN03_02'].where(df_final['FS01'] ==1, 0) * 0.7 + df_final['AN04_02'].where(df_final['FS01'] ==1, 0) * 0.1 + df_final['AN02_02'].where(df_final['FS01'] ==2, 0) * 0.2 + df_final['AN03_02'].where(df_final['FS01'] ==2, 0) * 0.5 + df_final['AN04_02'].where(df_final['FS01'] ==2, 0) * 0.3  + df_final['AN01_02'].where(df_final['FS01'] ==3, 0)

df_final['RISK_TOTAL']= df_final['AN02_01'].where(df_final['FS01'] ==1, 0) * 0.2 + df_final['AN03_01'].where(df_final['FS01'] ==1, 0) * 0.7 + df_final['AN04_01'].where(df_final['FS01'] ==1, 0) * 0.1 + df_final['AN02_01'].where(df_final['FS01'] ==2, 0) * 0.2 + df_final['AN03_01'].where(df_final['FS01'] ==2, 0) * 0.5 + df_final['AN04_01'].where(df_final['FS01'] ==2, 0) * 0.3 + df_final['AN01_01'].where(df_final['FS01'] ==3, 0)

#### SAVE_TOTAL and RISK_TOTAL have to sum up to 100

In [80]:
df_final

,FS01_CP,FS01,SD03,SD05,SD11,SD11_10,SD14,SD14_08,SD16,MZ03_01,...,RI08,RI09,RI10,RI11,RI12,RI13,TIME_SUM,FINISHED,SAVE_TOTAL,RISK_TOTAL
0,0,1,4,2,8,NaN,3,NaN,7,4,...,2,1,2,1,1,2,215,1,53.8,46.2
1,0,2,11,2,8,NaN,6,NaN,11,1,...,3,1,2,2,2,1,492,1,80.0,20.0
2,0,3,3,1,7,NaN,3,NaN,4,5,...,2,1,2,1,1,1,299,1,72.0,28.0
3,1,3,3,1,8,NaN,3,NaN,5,4,...,3,2,2,4,1,3,391,1,50.0,50.0
4,1,2,5,2,8,NaN,4,NaN,6,2,...,1,1,1,3,1,2,566,1,50.0,50.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154,60,1,3,2,8,NaN,3,NaN,5,4,...,2,1,1,1,1,1,242,1,95.0,5.0
155,60,3,4,1,8,NaN,3,NaN,1,3,...,3,1,2,1,1,2,361,1,70.0,30.0
156,61,2,4,2,8,NaN,4,NaN,7,4,...,2,1,2,2,2,2,232,1,65.0,35.0
157,61,3,7,1,8,NaN,4,NaN,8,4,...,1,1,2,1,1,1,330,1,80.0,20.0


#### Replace missing values of the 13-Item Risk Tolerance Scale with 1 (There are only very few)

In [81]:
df_final.loc[df_final['RI01'] == -9, 'RI01'] = 1
df_final.loc[df_final['RI02'] == -9, 'RI02'] = 1
df_final.loc[df_final['RI03'] == -9, 'RI03'] = 1
df_final.loc[df_final['RI04'] == -9, 'RI04'] = 1
df_final.loc[df_final['RI05'] == -9, 'RI05'] = 1
df_final.loc[df_final['RI06'] == -9, 'RI06'] = 1
df_final.loc[df_final['RI07'] == -9, 'RI07'] = 1
df_final.loc[df_final['RI08'] == -9, 'RI08'] = 1
df_final.loc[df_final['RI09'] == -9, 'RI09'] = 1
df_final.loc[df_final['RI10'] == -9, 'RI10'] = 1
df_final.loc[df_final['RI11'] == -9, 'RI11'] = 1
df_final.loc[df_final['RI12'] == -9, 'RI12'] = 1
df_final.loc[df_final['RI13'] == -9, 'RI13'] = 1

### Create a Column with the total sum of the 13-Item Risk Tolerance Scale

In [82]:
df_final["RISK_TOLERANCE_TOTAL"] = df_final['RI01'] + df_final['RI02'] +df_final['RI03'] +df_final['RI04'] +df_final['RI05'] +df_final['RI06'] +df_final['RI07'] +df_final['RI08'] +df_final['RI09']+ df_final['RI10'] +df_final['RI11']+df_final['RI12']+df_final['RI13']

### Replace Missing values with neutral answer (again only very few missing)

In [83]:
df_final.loc[df_final['MZ03_01'] == -9, 'MZ03_01'] = 3
df_final.loc[df_final['MZ03_02'] == -9, 'MZ03_02'] = 3
df_final.loc[df_final['MZ03_03'] == -9, 'MZ03_03'] = 3
df_final.loc[df_final['MZ03_04'] == -9, 'MZ03_04'] = 3
df_final.loc[df_final['MZ03_05'] == -9, 'MZ03_05'] = 3
df_final.loc[df_final['MZ03_06'] == -9, 'MZ03_06'] = 3
df_final.loc[df_final['VR01_01'] == -9, 'VR01_01'] = 3

#### Number of valid cases

In [84]:
len(df_final)

159

In [85]:
df_final

,FS01_CP,FS01,SD03,SD05,SD11,SD11_10,SD14,SD14_08,SD16,MZ03_01,...,RI09,RI10,RI11,RI12,RI13,TIME_SUM,FINISHED,SAVE_TOTAL,RISK_TOTAL,RISK_TOLERANCE_TOTAL
0,0,1,4,2,8,NaN,3,NaN,7,4,...,1,2,1,1,2,215,1,53.8,46.2,24
1,0,2,11,2,8,NaN,6,NaN,11,1,...,1,2,2,2,1,492,1,80.0,20.0,28
2,0,3,3,1,7,NaN,3,NaN,4,5,...,1,2,1,1,1,299,1,72.0,28.0,24
3,1,3,3,1,8,NaN,3,NaN,5,4,...,2,2,4,1,3,391,1,50.0,50.0,34
4,1,2,5,2,8,NaN,4,NaN,6,2,...,1,1,3,1,2,566,1,50.0,50.0,24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154,60,1,3,2,8,NaN,3,NaN,5,4,...,1,1,1,1,1,242,1,95.0,5.0,22
155,60,3,4,1,8,NaN,3,NaN,1,3,...,1,2,1,1,2,361,1,70.0,30.0,23
156,61,2,4,2,8,NaN,4,NaN,7,4,...,1,2,2,2,2,232,1,65.0,35.0,27
157,61,3,7,1,8,NaN,4,NaN,8,4,...,1,2,1,1,1,330,1,80.0,20.0,17


### Save to excel file

In [86]:
df_final.to_excel("data_masterstudie_filtered.xlsx")